# Code used to store Model Performances

## Setting up schema and databases

In [ ]:
create schema if not exists model_results_schema;

create table if not exists model_results_schema.confusion_matrix(
    id int autoincrement primary key,
    true_positive int,
    true_negative int, 
    false_positive int, 
    false_negative int,
    create_at timestamp default current_timestamp
);

create table if not exists model_results_schema.model_performance (
    model_name string,
    accuracy float,
    recall float,
    confusion_matrix_id int,
    training_time float,
    prediction_time float,
    parameters string,
    coefficients string, 
    intercept float,
    notes string, 
    create_at timestamp default current_timestamp
);

### Check results

In [ ]:
select * from model_results_schema.model_performance;
--select * from model_results_schema.confusion_matrix

## Python method to store results

Used by copying over into model notebooks. 

In [ ]:
import json

def record_performance(true_positive, true_negative, false_positive, false_negative, model_name, accuracy, recall, training_time, prediction_time, 
         parameters, coefficients, intercept, notes):
    ## NB: Make sure the name makes the model easily identifiable. 

    confusion_matrix_insert_sql = f"""
        insert into model_results_schema.confusion_matrix
        (true_positive, true_negative, false_positive, false_negative)
        values
        ({true_positive}, {true_negative}, {false_positive}, {false_negative});
    """
    
    session.sql(confusion_matrix_insert_sql).collect()

    last_id_sql = """
        select id
        from model_results_schema.confusion_matrix
        order by create_at desc
        limit 1;
    """ 
    confusion_matrix_id = session.sql(last_id_sql).collect()
    confusion_matrix_id = confusion_matrix_id[0]['ID']

    # These two need to be on a string format. 
    coefficients = ', '.join(map(str, coefficients))
    parameters = json.dumps(parameters)
    
    # Insert data into the model_performance table
    session.sql(f"""
        insert into model_results_schema.model_performance
            (model_name, accuracy, recall, confusion_matrix_id,
            training_time, prediction_time, parameters, coefficients,
            intercept, notes)
        values
            ('{model_name}', {accuracy}, {recall}, {confusion_matrix_id}, {training_time}, {prediction_time}, 
            '{parameters}', '{coefficients}', {intercept}, '{notes}');
    """).collect()
    
    return "success"


## Download Results

To download the results as a .csv file follwo the instructions below.

First set up a stage and copy the relevant table into the stage. 

In [ ]:
use schema model_results_schema;

CREATE OR REPLACE STAGE model_results;

COPY INTO @model_results/performance_3.csv
FROM model_performance
FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY='"', COMPRESSION = NONE)
OVERWRITE = TRUE
;

COPY INTO @model_results/confusion_matrix_3.csv
FROM confusion_matrix
FILE_FORMAT = (TYPE = 'CSV', FIELD_OPTIONALLY_ENCLOSED_BY='"', COMPRESSION = NONE)
OVERWRITE = TRUE
;

On your local machine run the following code and you should have a .csv file added to your directory.

In [ ]:
    from snowflake.snowpark import Session
    
    
    session = Session.builder.configs({
        'user': ,
        'password': ,
        'account': ,
        'database': 'ml',
        'schema': 'model_results_schema'
    }).create()
    
    
    local_path = ''
    stage_path = '@model_results/performance_3.csv'
    
    session.file.get(stage_path, local_path)
    
    
    print("File downloaded to:", local_path)
    
    session.close()
